<a href="https://colab.research.google.com/github/DipeshChothwe/NLP/blob/master/Topic_modeling_for_news_categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install news-fetch

     |████████████████████████████████| 911kB 3.8MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 215kB 15.7MB/s 
     |████████████████████████████████| 245kB 20.0MB/s 
     |████████████████████████████████| 245kB 19.3MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 3.0MB 23.8MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 225kB 46.8MB/s 
     |████████████████████████████████| 983kB 41.3MB/s 
     |████████████████████████████████| 40kB 5.1MB/s 
     |████████████████████████████████| 3.3MB 47.9MB/s 
     |████████████████████████████████| 7.4MB 43.6MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 81kB 7.7MB/s 
     |████████████████████████████████| 3.1MB 42.3MB/s 
     |████████████████████████████████| 235kB 44.8MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |█

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# **Data import**

Data was imported from the already available dataset for news article collection: https://www.kaggle.com/rmisra/news-category-dataset?rvi=1

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

news = pd.read_json('/content/drive/My Drive/NLP/News_Category_Dataset_v2.json', lines=True)

In [5]:
news.describe()

,category,headline,authors,link,short_description,date
count,200853,200853,200853,200853,200853,200853
unique,41,199344,27993,200812,178353,2309
top,POLITICS,Sunday Roundup,,https://www.huffingtonpost.comhttp://www.nytim...,,2013-01-17 00:00:00
freq,32739,90,36620,2,19712,100
first,NaN,NaN,NaN,NaN,NaN,2012-01-28 00:00:00
last,NaN,NaN,NaN,NaN,NaN,2018-05-26 00:00:00


In [6]:
news.groupby('category').describe()

headline         ...       date           
                  count unique  ...      first       last
category                        ...                      
ARTS               1509   1509  ... 2014-04-18 2017-01-28
ARTS & CULTURE     1339   1338  ... 2015-07-01 2018-01-29
BLACK VOICES       4528   4519  ... 2012-01-29 2018-05-25
BUSINESS           5937   5925  ... 2012-01-28 2018-05-24
COLLEGE            1144   1144  ... 2014-04-18 2018-02-07
COMEDY             5175   5118  ... 2012-01-28 2018-05-25
CRIME              3405   3400  ... 2012-02-11 2018-05-26
CULTURE & ARTS     1030   1010  ... 2012-01-28 2014-04-13
DIVORCE            3426   3420  ... 2012-01-30 2014-04-17
EDUCATION          1004   1004  ... 2014-04-18 2018-05-17
ENTERTAINMENT     16058  16036  ... 2012-01-28 2018-05-26
ENVIRONMENT        1323   1251  ... 2012-01-28 2014-04-13
FIFTY              1401   1401  ... 2014-04-18 2017-02-05
FOOD & DRINK       6226   6221  ... 2012-01-30 2014-04-17
GOOD NEWS          1398   1398  ... 2014-04-22 2017-08-09
GREEN              2622   2617  ... 2014-04-18 2018-01-17
HEALTHY LIVING     6694   6684  ... 2014-04-18 2018-01-16
HOME & LIVING      4195   4089  ... 2012-01-28 2014-04-17
IMPACT             3459   3455  ... 2012-01-28 2018-05-26
LATINO VOICES      1129   1129  ... 2014-04-22 2018-05-22
MEDIA              2815   2813  ... 2014-04-18 2018-05-24
MONEY              1707   1705  ... 2012-02-29 2014-04-17
PARENTING          8677   8626  ... 2012-01-28 2014-04-17
PARENTS            3955   3890  ... 2014-04-18 2018-02-06
POLITICS          32739  32616  ... 2014-04-18 2018-05-26
QUEER VOICES       6314   6305  ... 2012-01-28 2018-05-25
RELIGION           2556   2548  ... 2014-04-18 2018-05-23
SCIENCE            2178   2176  ... 2012-01-28 2018-05-23
SPORTS             4884   4883  ... 2012-01-28 2018-05-24
STYLE              2254   2246  ... 2014-04-18 2018-01-19
STYLE & BEAUTY     9649   9165  ... 2012-01-28 2014-04-17
TASTE              2096   2095  ... 2014-04-18 2018-01-16
TECH               2082   2027  ... 2012-01-28 2018-05-24
THE WORLDPOST      3664   3662  ... 2015-07-09 2017-10-13
TRAVEL             9887   9862  ... 2012-01-28 2018-05-24
WEDDINGS           3651   3592  ... 2012-01-29 2014-04-17
WEIRD NEWS         2670   2670  ... 2014-04-19 2018-05-26
WELLNESS          17827  17778  ... 2012-01-28 2014-04-17
WOMEN              3490   3375  ... 2014-04-18 2018-05-25
WORLD NEWS         2177   2174  ... 2017-04-09 2018-05-26
WORLDPOST          2579   2579  ... 2014-04-18 2017-02-12

[41 rows x 30 columns]

In [7]:
print("missing values from msg:")
print(news.isnull().sum())



missing values from msg:
category             0
headline             0
authors              0
link                 0
short_description    0
date                 0
dtype: int64


In [8]:
news.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [9]:
news['category'].unique()

array(['CRIME', 'ENTERTAINMENT', 'WORLD NEWS', 'IMPACT', 'POLITICS',
       'WEIRD NEWS', 'BLACK VOICES', 'WOMEN', 'COMEDY', 'QUEER VOICES',
       'SPORTS', 'BUSINESS', 'TRAVEL', 'MEDIA', 'TECH', 'RELIGION',
       'SCIENCE', 'LATINO VOICES', 'EDUCATION', 'COLLEGE', 'PARENTS',
       'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE', 'HEALTHY LIVING',
       'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST', 'FIFTY', 'ARTS',
       'WELLNESS', 'PARENTING', 'HOME & LIVING', 'STYLE & BEAUTY',
       'DIVORCE', 'WEDDINGS', 'FOOD & DRINK', 'MONEY', 'ENVIRONMENT',
       'CULTURE & ARTS'], dtype=object)

In [10]:
newnews = news.loc[news['category'].isin(['ENTERTAINMENT','POLITICS','HEALTHY LIVING','SPORTS','BUSINESS'])]

In [11]:
newnews['category'].unique()

array(['ENTERTAINMENT', 'POLITICS', 'SPORTS', 'BUSINESS',
       'HEALTHY LIVING'], dtype=object)

# **Data preprocessing**



In [12]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [13]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

        
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove tokens that are less than length 2
    text = [t for t in text if len(t) > 1]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # join all
    text = " ".join(text)
    return(text)

In [14]:
# clean text data
newnews["headline"] =newnews["headline"].apply(lambda x: clean_text(x))
newnews["short_description"] = newnews["short_description"].apply(lambda x: clean_text(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# **Feature engineering**

In [15]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def feature_engineering(column):
  from sklearn.feature_extraction.text import TfidfVectorizer
  from nltk.sentiment.vader import SentimentIntensityAnalyzer

  sid = SentimentIntensityAnalyzer()
  tf_vec = TfidfVectorizer()

  feature = tf_vec.fit_transform(column)
  t = {}
  train_data = pd.DataFrame(t)
  train_data["sentiments"] = column.apply(lambda x: sid.polarity_scores(x))
  train_data = pd.concat([train_data.drop(['sentiments'], axis=1), train_data['sentiments'].apply(pd.Series)], axis=1)
  train_data['compound'] = train_data['compound'] + 1
  from scipy.sparse import hstack
  featuresfinal = hstack((feature,np.array(train_data['compound'])[:,None]))

  return featuresfinal

In [16]:
f1 = feature_engineering(newnews["headline"])
f2 = feature_engineering(newnews["short_description"])

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [17]:
from scipy.sparse import hstack
featuresfinal = hstack((f1,np.array(f2).all()))

# **Model**

In [18]:
features_train, features_test, labels_train, labels_test = train_test_split(featuresfinal, newnews['category'], test_size=0.3, random_state=100)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

In [20]:

svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=100)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=100)
abc = AdaBoostClassifier(n_estimators=62, random_state=100)
bc = BaggingClassifier(n_estimators=9, random_state=100)
etc = ExtraTreesClassifier(n_estimators=9, random_state=100)


clfs = {'KN' : knc, 'mnb' : mnb, 'DT': dtc, 'LR': lrc, 'RF': rfc, 'AdaBoost': abc, 'BgC': bc, 'ETC': etc}
#clfs = {'LR': lrc}

In [21]:
def train_classifier(clf, feature_train, labels_train):    
    clf.fit(feature_train, labels_train)

def predict_labels(clf, features):
    return (clf.predict(features))

In [22]:
pred_scores = []
for k,v in clfs.items():
  print(k,v)
  train_classifier(v, features_train, labels_train)
  pred = predict_labels(v,features_test)
  pred_scores.append((k, [accuracy_score(labels_test,pred)]))

KN KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=49, p=2,
                     weights='uniform')
mnb MultinomialNB(alpha=0.2, class_prior=None, fit_prior=True)
DT DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=100, splitter='best')
LR LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   wa

In [23]:

Score_sheet = pd.DataFrame(pred_scores)
Score_sheet

,0,1
0,KN,[0.6889011762340405]
1,mnb,[0.8517643510606213]
2,DT,[0.7026239067055393]
3,LR,[0.8599577762139339]
4,RF,[0.8115512214738112]
5,AdaBoost,[0.6092791796521564]
6,BgC,[0.7542475118126069]
7,ETC,[0.8055695184477732]
